In [ ]:
import feedparser
import dateutil.parser
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import mongo_utils

In [ ]:
dbname = mongo_utils.get_database()

In [ ]:
urls = ['https://rss.frognews.bg/',
    ]
feeds = [feedparser.parse(url)['entries'] for url in urls]
feed = [item for feed in feeds for item in feed]

In [ ]:
for newslink in feed:
  try:
    #print(newslink)
    nlink=newslink["link"]
    print(nlink)
    date_posted=newslink["published"]
    news_title=newslink["title"]
    text=newslink["summary"]
    page = requests.get(nlink)
    data = page.text
    soup = BeautifulSoup(data, 'html.parser')
    text_highlight= soup.find('div', class_='article-short-text brc').get_text()
    text_body= soup.find('div', class_='article-full-text').get_text()
    text=(text_highlight + text_body)
    text=text.replace('\n',' ').replace('\xa0', ' ').replace('\r', '').replace('\t', '')
    news_payload = {
    "date_posted" : date_posted,
    "news_link" : nlink,
    "news_text" : text,
    "news_title" : news_title
    }
    print(news_payload)
    collection_name = dbname["frognews_scrape_1"]
    collection_name.insert_many([news_payload])
  except:
    pass